In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Задание 1

Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока).

### Анализ и обработка данных

In [11]:
# Датасэт из первой задании
reg_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-ikramov/shared/problem1-reg_data.csv', sep=';')
reg_data['reg_ts'] = pd.to_datetime(reg_data['reg_ts'], unit='s').dt.date
reg_data.head()

,reg_ts,uid
0,1998-11-18,1
1,1999-07-22,2
2,2000-01-13,3
3,2000-05-28,4
4,2000-09-16,5


In [12]:
reg_data.isna().sum()

reg_ts    0
uid       0
dtype: int64

In [13]:
# Датасэт из первой задании
auth_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-ikramov/shared/problem1-auth_data.csv', sep=';')
auth_data['auth_ts'] = pd.to_datetime(auth_data['auth_ts'], unit='s').dt.date
auth_data.head()

,auth_ts,uid
0,1998-11-18,1
1,1999-07-22,2
2,1999-07-25,2
3,1999-07-31,2
4,1999-08-05,2


In [14]:
auth_data.isna().sum()

auth_ts    0
uid        0
dtype: int64

##### Join auth_data и reg_data

In [15]:
df = pd.merge(auth_data, reg_data, on='uid', how='left')

new_cols = ['uid', 'reg_ts', 'auth_ts']
df = df[new_cols]
df = df.rename(columns=({'uid': 'player_id', 'reg_ts': 'registration_date', 'auth_ts': 'active_date'}))
df.head()

,player_id,registration_date,active_date
0,1,1998-11-18,1998-11-18
1,2,1999-07-22,1999-07-22
2,2,1999-07-22,1999-07-25
3,2,1999-07-22,1999-07-31
4,2,1999-07-22,1999-08-05


### Функция, которая считаeт retention игроков

In [16]:
import pandas as pd

def calculate_retention(df, start_reg_date, end_reg_date, retention_days):
    """
    Рассчитывает retention пользователей, зарегистрированных в определённый период,
    и рассчитывает retention на указанный период (в днях) после регистрации.
    
    :param df: DataFrame, содержащий колонки player_id, registration_date, active_date
    :param start_reg_date: начальная дата регистрации игроков (в формате 'YYYY-MM-DD')
    :param end_reg_date: конечная дата регистрации игроков (в формате 'YYYY-MM-DD')
    :param retention_days: количество дней после регистрации, на которые рассчитывается retention (например, 7 или 30 дней)
    
    :return: DataFrame с расчетом retention на выбранный период
    """
    
    # Преобразуем start_reg_date и end_reg_date в формат datetime внутри функции
    start_reg_date = pd.to_datetime(start_reg_date)
    end_reg_date = pd.to_datetime(end_reg_date)
    
    # Фильтруем пользователей по периоду регистрации между start_reg_date и end_reg_date
    df_filtered = df[(df['registration_date'] >= start_reg_date) & (df['registration_date'] <= end_reg_date)]
    
    # Добавляем колонку с разницей между датой активности и датой регистрации в днях
    df_filtered['days_since_registration'] = (df_filtered['active_date'] - df_filtered['registration_date']).dt.days
    
    # Фильтруем по указанному периоду retention (retention_days)
    df_filtered = df_filtered[df_filtered['days_since_registration'] <= retention_days]
    
    # Создаем таблицу: дата регистрации + разница в днях
    cohort_data = df_filtered.groupby(['registration_date', 'days_since_registration']).agg({
        'player_id': 'nunique'
    }).reset_index()
    
    # Создаем таблицу: дата регистрации + общее количество пользователей, зарегистрировавшихся в этот день
    registered_players = df_filtered.groupby('registration_date').agg({
        'player_id': 'nunique'
    }).rename(columns={'player_id': 'total_registered'}).reset_index()
    
    # Объединяем данные
    retention_data = pd.merge(cohort_data, registered_players, on='registration_date')
    
    # Рассчитываем retention в процентах
    retention_data['retention_rate'] = round((retention_data['player_id'] / retention_data['total_registered']) * 100, 2)
    
    # Преобразуем данные для более удобного отображения (pivot)
    retention_pivot = retention_data.pivot(index='registration_date', 
                                           columns='days_since_registration', 
                                           values='retention_rate').fillna(0)
    
    return retention_pivot

#### Рассчитаем retention для пользователей, зарегистрированных с 2020-09-01 по 2020-09-08, на период 21 дней после регистрации

In [17]:
retention = calculate_retention(df, start_reg_date='2020-09-01', end_reg_date='2020-09-08', retention_days=21)
pd.set_option('display.max_columns', None)
retention

days_since_registration,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
registration_date,,,,,,,,,,,,,,,,,,,,,,
2020-09-01,100.0,2.02,4.10,4.22,4.60,6.43,5.61,6.43,4.10,5.42,5.11,5.36,4.85,5.49,4.29,4.41,4.35,5.04,4.16,4.35,3.59,4.35
2020-09-02,100.0,2.52,3.90,4.66,5.67,6.23,7.12,6.30,4.85,4.09,5.67,5.48,5.48,4.35,4.97,5.29,4.16,4.85,3.90,3.72,4.66,2.64
2020-09-03,100.0,2.33,5.41,4.71,6.54,6.98,8.55,6.35,5.47,6.29,6.35,5.53,5.85,5.97,5.47,5.34,4.40,4.84,4.65,5.47,2.39,0.00
2020-09-04,100.0,2.01,3.64,3.64,5.15,6.40,7.28,4.96,5.34,4.33,5.65,5.52,4.33,5.59,4.39,4.52,4.52,4.27,4.77,2.64,0.00,0.00
2020-09-05,100.0,2.76,3.95,4.64,5.45,5.89,7.46,4.89,5.08,5.45,6.02,5.20,4.57,5.39,4.51,5.14,4.45,4.76,2.51,0.00,0.00,0.00
2020-09-06,100.0,3.13,4.32,5.44,5.63,5.88,6.82,6.07,5.82,5.50,4.88,4.75,5.38,5.19,5.50,4.57,4.82,2.56,0.00,0.00,0.00,0.00
2020-09-07,100.0,2.94,4.25,5.37,5.06,6.37,7.31,5.25,4.12,5.62,6.00,5.68,4.56,5.06,4.25,5.06,3.37,0.00,0.00,0.00,0.00,0.00
2020-09-08,100.0,1.75,3.93,3.93,4.86,7.61,7.54,6.61,5.17,5.17,6.55,5.36,5.92,5.17,4.74,3.05,0.00,0.00,0.00,0.00,0.00,0.00
